In [ ]:
!pip install pyLDAvis
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import TfidfModel
from gensim.models import CoherenceModel
from gensim.parsing.porter import PorterStemmer
import pyLDAvis
import pyLDAvis.gensim_models
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import string
from pprint import pprint
import spacy
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/jsonschema/compat.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import MutableMapping, Sequence  # noqa
/usr/local/lib/python3.7/dist-packages/jsonschema/compat.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import MutableMapping, Sequence  # noqa


Importing the datafile using pandas

In [ ]:
dataset = pd.read_json('/content/assignment_dump.json', orient='columns')

In [ ]:
dataset.head(10)

,abstract,authors,n_citation,references,title,venue,year,id
0,This paper describes the design and implementa...,"[Gareth Beale, Graeme Earl]",50,"[10482dd3-4642-4193-842f-85f3b70fcf65, 3133714...",A methodology for the physically accurate visu...,visual analytics science and technology,2011,001c58d3-26ad-46b3-ab3a-c1e557d16821
1,This article applied GARCH model instead AR or...,"[Altaf Hossain, Faisal Zaman, Mohammed Nasser,...",50,"[2d84c0f2-e656-4ce7-b018-90eda1c132fe, a083a1b...","Comparison of GARCH, Neural Network and Suppor...",pattern recognition and machine intelligence,2009,001c8744-73c4-4b04-9364-22d31a10dbf1
2,"Recently, Bridges and Reich introduced the con...","[Alvaro L. Islas, Constance M. Schober]",50,[],Multisymplectic Spectral Methods for the Gross...,international conference on conceptual structures,2002,00bcf2d5-1592-46b0-81fd-933f90b5ecca
3,Most applications of the abstract interpretati...,"[Patrick Cousot, Radhia Cousot]",50,"[6e8a3ec3-9a99-4fac-ac4d-f8d1bf0fa208, 7bb71af...",Relational Abstract Interpretation of Higher O...,,1991,00c85316-bddf-4bcb-93f5-097adadd73c2
4,This paper focuses on knowledge engineering fo...,"[Efthymios Alepis, Maria Virvou, Katerina Kaba...",50,"[522bea13-bd45-4918-9478-e87ae2f039f7, 60507d4...",Knowledge Engineering for Affective Bi-Modal I...,joint conference on knowledge-based software e...,2008,00dd5ece-1339-4cb6-a08e-43742ae2dac6
5,,"[Abdullah Alsubaie, Antonio Di Pietro, Jose R....",6,"[0293844d-7244-4562-a6ca-d84bcd9c10a5, 14cd3cf...",A Platform for Disaster Response Planning with...,,2013,00e3940b-201b-40ba-bd20-dbdaf26ad324
6,Xax is a browser plugin model that enables dev...,"[John R. Douceur, Jeremy Elson, Jon Howell, Ja...",50,"[0abc9de7-e047-44fc-998d-4bf02b9bc9ab, 2c9ebc3...",Leveraging legacy code to deploy desktop appli...,operating systems design and implementation,2008,010d4ce9-0279-4166-ae73-14551ded6404
7,"In recent years, most of mobile phones have a ...","[Hiroshi Furukawa, Yutaka Nakamura]",7,"[b2f0e0d3-0071-4076-921e-f93f30160c1c, f030524...",A pedestrian navigation method for user's safe...,international conference on human computer int...,2013,010d9907-45ef-459a-a293-8e1f41cef2b7
8,Previous language modeling approaches to infor...,"[Carmen Fernández-Daza Álvarez, Philippe Langl...",50,"[00515e82-3da6-497a-83d3-459e93247123, 0cd820e...",Word pairs in language modeling for informatio...,,2004,012b88ae-a763-45d6-8f19-2ec9ff739d5f
9,,"[Prudence W. H. Wong, Fencol C. C. Yung, Mihai...",6,"[08784bb3-09b9-4bd2-a191-2606f3b3a8b8, 16bf10e...",An 8/3 Lower Bound for Online Dynamic Bin Packing,international symposium on algorithms and comp...,2012,01546472-0ac9-47fa-acca-ffcb7fc99097


Performing preprocessing steps on our data

In [ ]:
dataset["abstract"]=dataset["abstract"].fillna("")
dataset["title"]=dataset["title"].fillna("")
dataset["venue"]=dataset["venue"].fillna("")
dst1=dataset["abstract"].tolist()
dst2=dataset["title"].tolist()
dst3=dataset["venue"].tolist()
dst1=["{} ".format(ele) for ele in np.squeeze(dst1)]
dst2=["{} ".format(ele) for ele in np.squeeze(dst2)]
dst3=["{} ".format(ele) for ele in np.squeeze(dst3)]
dst=np.char.add(dst1, dst2)
dst=np.char.add(dst, dst3)

Here we have used the information provided to us by abstract, title and venue, and concatenated all 3 to form variable dst, where each row is a single article's abstract, title and venue.

In [ ]:
print(dst[0:10])

["This paper describes the design and implementation of a methodology for the visualisation and hypothetical virtual reconstruction of Roman polychrome statuary for research purposes. The methodology is intended as an attempt to move beyond visualisations which are simply believable towards a more physically accurate approach. Accurate representations of polychrome statuary have great potential utility both as a means of illustrating existing interpretations and as a means of testing and revising developing hypotheses. The goal of this methodology is to propose a pipeline which incorporates a high degree of physical accuracy whilst also being practically applicable in a conventional archaeological research setting. The methodology is designed to allow the accurate visualisation of surviving objects and colourants as well as providing reliable methods for the hypothetical reconstruction of elements which no longer survive. The process proposed here is intended to limit the need for spec

Removing stop words from dst, performing tokenization and naming it dtst.

In [ ]:
dtst=[[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in dst]

Performing lemmatizaion on dtst.

In [ ]:
print(dtst[0:5])

[['paper', 'describes', 'design', 'implementation', 'methodology', 'visualisation', 'hypothetical', 'virtual', 'reconstruction', 'roman', 'polychrome', 'statuary', 'research', 'purposes', 'methodology', 'intended', 'attempt', 'move', 'beyond', 'visualisations', 'simply', 'believable', 'towards', 'physically', 'accurate', 'approach', 'accurate', 'representations', 'polychrome', 'statuary', 'great', 'potential', 'utility', 'means', 'illustrating', 'existing', 'interpretations', 'means', 'testing', 'revising', 'developing', 'hypotheses', 'goal', 'methodology', 'propose', 'pipeline', 'incorporates', 'high', 'degree', 'physical', 'accuracy', 'whilst', 'also', 'practically', 'applicable', 'conventional', 'archaeological', 'research', 'setting', 'methodology', 'designed', 'allow', 'accurate', 'visualisation', 'surviving', 'objects', 'colourants', 'well', 'providing', 'reliable', 'methods', 'hypothetical', 'reconstruction', 'elements', 'longer', 'survive', 'process', 'proposed', 'intended', 'l

In [ ]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
nlp = spacy.load('en', disable=['parser', 'ner'])

In [ ]:
dtst=lemmatization(dtst)

In [ ]:
print(dtst[0])

['paper', 'describe', 'design', 'implementation', 'methodology', 'visualisation', 'hypothetical', 'virtual', 'reconstruction', 'research', 'purpose', 'methodology', 'intend', 'attempt', 'move', 'visualisation', 'simply', 'believable', 'physically', 'accurate', 'approach', 'accurate', 'representation', 'polychrome', 'statuary', 'great', 'potential', 'utility', 'mean', 'illustrate', 'exist', 'interpretation', 'mean', 'test', 'revise', 'develop', 'hypothesis', 'goal', 'methodology', 'propose', 'pipeline', 'incorporate', 'high', 'degree', 'physical', 'accuracy', 'also', 'practically', 'applicable', 'conventional', 'archaeological', 'research', 'set', 'methodology', 'design', 'allow', 'accurate', 'visualisation', 'surviving', 'object', 'colourant', 'well', 'provide', 'reliable', 'method', 'hypothetical', 'reconstruction', 'element', 'longer', 'survive', 'process', 'propose', 'intend', 'limit', 'need', 'specialist', 'recording', 'equipment', 'utilise', 'exist', 'datum', 'datum', 'collect', '

Creating a dictionary of our preprocessed data and using it to create a bag-of-words corpus

In [ ]:
id2word = corpora.Dictionary(dtst)
texts=dtst
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[:1])

[[(0, 1), (1, 4), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 1), (20, 1), (21, 2), (22, 1), (23, 1), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 1), (30, 1), (31, 2), (32, 1), (33, 1), (34, 2), (35, 1), (36, 1), (37, 1), (38, 1), (39, 2), (40, 1), (41, 1), (42, 1), (43, 2), (44, 1), (45, 5), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 2), (54, 1), (55, 2), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 2), (62, 1), (63, 1), (64, 2), (65, 1), (66, 1), (67, 1), (68, 2), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 2), (77, 2), (78, 1), (79, 1), (80, 1), (81, 2), (82, 1), (83, 2), (84, 1), (85, 1), (86, 1), (87, 1), (88, 4), (89, 1), (90, 1)]]


Printing the corpus to show combinations of words and frequencies.

In [ ]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('accuracy', 1),
  ('accurate', 4),
  ('allow', 1),
  ('also', 1),
  ('analytic', 1),
  ('applicable', 1),
  ('approach', 1),
  ('archaeological', 1),
  ('area', 1),
  ('attempt', 1),
  ('available', 1),
  ('believable', 1),
  ('case', 1),
  ('collect', 1),
  ('colourant', 1),
  ('context', 1),
  ('conventional', 1),
  ('datum', 2),
  ('degree', 1),
  ('demonstrate', 1),
  ('describe', 1),
  ('design', 2),
  ('develop', 1),
  ('discover', 1),
  ('element', 1),
  ('equipment', 1),
  ('exist', 2),
  ('female', 1),
  ('goal', 1),
  ('great', 1),
  ('head', 1),
  ('herculaneum', 2),
  ('high', 1),
  ('hypothesis', 1),
  ('hypothetical', 2),
  ('illustrate', 1),
  ('implement', 1),
  ('implementation', 1),
  ('incorporate', 1),
  ('intend', 2),
  ('interpretation', 1),
  ('limit', 1),
  ('longer', 1),
  ('mean', 2),
  ('method', 1),
  ('methodology', 5),
  ('move', 1),
  ('need', 1),
  ('object', 1),
  ('paint', 1),
  ('paper', 1),
  ('part', 1),
  ('physical', 1),
  ('physically', 2),
  

Number of branches will determine the number of clusters at each heirarchical level

In [ ]:
branches = 4

In [ ]:
branches2 = branches*branches

Here we have used the LDA (Latent Dirichlet Allocation) method for clustering 

In [ ]:
lda_model1 = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=branches, random_state=100, update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)

In [ ]:
pprint(lda_model1.print_topics())
doc_lda = lda_model1[corpus]

[(0,
  '0.022*"problem" + 0.011*"algorithm" + 0.011*"time" + 0.009*"show" + '
  '0.009*"use" + 0.009*"set" + 0.008*"model" + 0.008*"paper" + 0.007*"result" '
  '+ 0.007*"give"'),
 (1,
  '0.029*"network" + 0.012*"use" + 0.012*"system" + 0.012*"performance" + '
  '0.010*"propose" + 0.009*"time" + 0.009*"control" + 0.009*"base" + '
  '0.008*"scheme" + 0.007*"communication"'),
 (2,
  '0.025*"system" + 0.013*"model" + 0.011*"use" + 0.011*"base" + 0.011*"user" '
  '+ 0.010*"information" + 0.010*"paper" + 0.009*"process" + '
  '0.008*"application" + 0.008*"service"'),
 (3,
  '0.021*"use" + 0.017*"method" + 0.015*"image" + 0.014*"base" + 0.013*"datum" '
  '+ 0.011*"propose" + 0.010*"model" + 0.010*"approach" + 0.009*"feature" + '
  '0.009*"result"')]


Analyzing performance of the models

In [ ]:
print('\nPerplexity: ', lda_model1.log_perplexity(corpus))
coherence_model_lda = CoherenceModel(model=lda_model1, texts=dtst, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.640504873674151

Coherence Score:  0.40279244492955313


In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model1, corpus, id2word)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.081971  0.220259       1        1  34.028941
0     -0.111053 -0.170716       2        1  25.409005
3     -0.084243 -0.035498       3        1  24.396824
1      0.277267 -0.014045       4        1  16.165231, topic_info=            Term          Freq         Total Category  logprob  loglift
110      network  26197.000000  26197.000000  Default  30.0000  30.0000
172      problem  31765.000000  31765.000000  Default  29.0000  29.0000
390        image  20919.000000  20919.000000  Default  28.0000  28.0000
44        method  32555.000000  32555.000000  Default  27.0000  27.0000
208         user  20018.000000  20018.000000  Default  26.0000  26.0000
..           ...           ...           ...      ...      ...      ...
50         paper   6017.114686  45577.337120   Topic4  -5.0111  -0.2025
116       result   5277.884083  32479.394966   Topic4  -5.1422   0.0052
117         show   4934.185374  27735.833089   Topic4  -5.2095   0.0958
152  application   4702.795196  26766.296839   Topic4  -5.2575   0.0833
17         datum   3958.805161  27438.072060   Topic4  -5.4298  -0.1137

[254 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
0         3  0.950017   accuracy
0         4  0.049720   accuracy
1247      1  0.999929   activity
718       1  0.999944      agent
473       2  0.788297  algorithm
...     ...       ...        ...
467       3  0.999803     vision
87        3  0.999719     visual
266       1  0.999916        web
703       4  0.999883   wireless
332       3  0.999824       word

[344 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 4, 2])

We create another level of heirarchy for clustering by using the data from bag-of-words of each article

In [ ]:
tstdc = [id2word.doc2bow(dtst[i]) for i in range(100000)]
vctrt = [lda_model1[tstdc[i]] for i in range(100000)]
print(vctrt[0])

([(0, 0.080858715), (1, 0.11892929), (2, 0.5050257), (3, 0.29518637)], [(0, [3, 1]), (1, [3, 1]), (2, [2, 3, 1, 0]), (3, [2, 3, 0, 1]), (4, [2, 0]), (5, [2, 0, 3, 1]), (6, [2, 3, 0]), (7, [3, 2]), (8, [2, 1, 3, 0]), (9, [2, 3, 1, 0]), (10, [2, 3, 1]), (11, [2]), (12, [2, 0, 3]), (13, [2, 3, 1]), (14, [2, 3, 1, 0]), (15, [2, 3, 0]), (16, [3, 1, 0]), (17, [3, 2, 1]), (18, [0, 2, 1, 3]), (19, [3, 2, 1, 0]), (20, [2, 3, 0]), (21, [2, 1, 0]), (22, [2, 3, 1, 0]), (23, [3, 2]), (24, [2, 0, 3, 1]), (25, [2, 1]), (26, [2, 3, 1, 0]), (27, [3]), (28, [2, 0]), (29, [2, 3, 1, 0]), (30, [3, 1]), (31, [2, 3, 1, 0]), (32, [1, 3, 2, 0]), (33, [3, 2, 0]), (34, [0]), (35, [2, 0, 3]), (36, [2, 1, 0]), (37, [2, 1, 0]), (38, [2, 3, 0]), (39, [2, 3]), (40, [3, 2, 0]), (41, [1, 2, 0, 3]), (42, [2, 1, 0]), (43, [3, 2, 0, 1]), (44, [3, 0]), (45, [2, 3, 0]), (46, [1, 2, 0]), (47, [2, 3, 1, 0]), (48, [3]), (49, [2]), (50, [2, 3, 1, 0]), (51, [2, 3, 0, 1]), (52, [2, 1]), (53, [1]), (54, [1, 3]), (55, [2, 3, 1, 0])

In [ ]:
print(vctrt[0][0])

[(0, 0.080858715), (1, 0.11892929), (2, 0.5050257), (3, 0.29518637)]


In [ ]:
abx = [vctrt[i][0] for i in range(100000)]
print (abx[0:10])

[[(0, 0.080858715), (1, 0.11892929), (2, 0.5050257), (3, 0.29518637)], [(0, 0.02434343), (1, 0.11095845), (2, 0.25153962), (3, 0.6131585)], [(0, 0.55737257), (1, 0.09218696), (2, 0.107057), (3, 0.24338348)], [(0, 0.9387306), (1, 0.013899789), (2, 0.02795466), (3, 0.019414986)], [(0, 0.032628868), (1, 0.02573743), (2, 0.75536066), (3, 0.1862731)], [(0, 0.1041947), (1, 0.17132084), (2, 0.6545024), (3, 0.06998208)], [(0, 0.104547955), (1, 0.22488087), (2, 0.6332762), (3, 0.037294995)], [(0, 0.16001098), (1, 0.27293032), (2, 0.41450843), (3, 0.15255024)], [(0, 0.18581048), (2, 0.071019165), (3, 0.73591197)], [(0, 0.6632068), (1, 0.053174023), (2, 0.2153057), (3, 0.06831345)]]


In [ ]:
reslt = [max(abx[k], key= lambda i : i[1])[0] for k in range(100000)]
print(reslt[0:10])

[2, 3, 0, 0, 2, 2, 2, 2, 3, 0]


In [ ]:
dtst1 = [[dtst[i] for i in range(100000) if reslt[i]==j] for j in range(branches)]
id2word2 = [corpora.Dictionary(dtst1[i]) for i in range(branches)]
corpus1 = [[id2word2[i].doc2bow(text) for text in dtst1[i]] for i in range(branches)]

In [ ]:
lda_model2 = [gensim.models.ldamodel.LdaModel(corpus=corpus1[i], id2word=id2word2[i], num_topics=branches, random_state=100, update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True) for i in range(branches)]

In [ ]:
for i in range(branches):
  pprint(lda_model2[i].print_topics())

[(0,
  '0.016*"language" + 0.011*"science" + 0.010*"code" + 0.010*"computer" + '
  '0.009*"show" + 0.008*"prove" + 0.008*"computation" + 0.008*"theory" + '
  '0.008*"proof" + 0.008*"complexity"'),
 (1,
  '0.027*"problem" + 0.017*"algorithm" + 0.016*"use" + 0.011*"base" + '
  '0.011*"time" + 0.010*"network" + 0.010*"propose" + 0.010*"optimization" + '
  '0.009*"result" + 0.009*"search"'),
 (2,
  '0.020*"problem" + 0.017*"graph" + 0.015*"time" + 0.012*"number" + '
  '0.012*"give" + 0.011*"set" + 0.010*"function" + 0.010*"show" + '
  '0.009*"bound" + 0.008*"result"'),
 (3,
  '0.024*"system" + 0.021*"model" + 0.016*"logic" + 0.014*"program" + '
  '0.013*"use" + 0.012*"base" + 0.010*"set" + 0.010*"paper" + 0.009*"rule" + '
  '0.008*"approach"')]
[(0,
  '0.026*"system" + 0.016*"use" + 0.012*"time" + 0.011*"application" + '
  '0.011*"performance" + 0.011*"datum" + 0.010*"design" + 0.009*"high" + '
  '0.008*"paper" + 0.007*"base"'),
 (1,
  '0.034*"attack" + 0.027*"key" + 0.026*"scheme" + 0.025

In [ ]:
print('\nPerplexity: ', lda_model2[1].log_perplexity(corpus1[1]))
coherence_model_lda = CoherenceModel(model=lda_model2[1], texts=dtst1[1], dictionary=id2word2[1], coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.572216589256754

Coherence Score:  0.4625621432487025


In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model2, corpus, id2word)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.081971  0.220259       1        1  34.028941
0     -0.111053 -0.170716       2        1  25.409005
3     -0.084243 -0.035498       3        1  24.396824
1      0.277267 -0.014045       4        1  16.165231, topic_info=            Term          Freq         Total Category  logprob  loglift
110      network  26197.000000  26197.000000  Default  30.0000  30.0000
172      problem  31765.000000  31765.000000  Default  29.0000  29.0000
390        image  20919.000000  20919.000000  Default  28.0000  28.0000
44        method  32555.000000  32555.000000  Default  27.0000  27.0000
208         user  20018.000000  20018.000000  Default  26.0000  26.0000
..           ...           ...           ...      ...      ...      ...
50         paper   6017.114686  45577.337120   Topic4  -5.0111  -0.2025
116       result   5277.884083  32479.394966   Topic4  -5.1422   0.0052
117         show   4934.185374  27735.833089   Topic4  -5.2095   0.0958
152  application   4702.795196  26766.296839   Topic4  -5.2575   0.0833
17         datum   3958.805161  27438.072060   Topic4  -5.4298  -0.1137

[254 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
0         3  0.950017   accuracy
0         4  0.049720   accuracy
1247      1  0.999929   activity
718       1  0.999944      agent
473       2  0.788297  algorithm
...     ...       ...        ...
467       3  0.999803     vision
87        3  0.999719     visual
266       1  0.999916        web
703       4  0.999883   wireless
332       3  0.999824       word

[344 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 4, 2])

In [ ]:
vector = [None]*100000
for j in range(100000):
  z=reslt[j]
  tdc = id2word2[z].doc2bow(dtst[j])
  vtr = lda_model2[z][tdc]
  vector[j] = vtr[0]

In [ ]:
reslt2 = [max(vector[k], key= lambda i : i[1])[0] for k in range(len(vector))]

In [ ]:
print(reslt2[0:100])

[3, 2, 2, 3, 3, 2, 3, 1, 1, 2, 1, 3, 3, 3, 0, 2, 3, 0, 1, 0, 3, 2, 3, 2, 2, 3, 3, 3, 3, 2, 3, 1, 3, 3, 2, 3, 2, 2, 2, 3, 1, 3, 1, 0, 0, 2, 3, 2, 2, 3, 0, 3, 2, 2, 2, 3, 1, 2, 3, 2, 1, 3, 2, 3, 3, 0, 3, 1, 3, 2, 3, 3, 0, 3, 2, 2, 3, 2, 3, 0, 1, 1, 3, 0, 0, 1, 2, 3, 3, 3, 1, 3, 3, 2, 3, 3, 3, 3, 0, 3]


In [ ]:
fin_res = branches*np.array(reslt) + np.array(reslt2)

In [ ]:
print(fin_res[0:100])

[11 14  2  3 11 10 11  9 13  2  5 15  3 11 12 14 11 12  9 12  3  2 11 14
  2 11 15 15 11  2 11  1 11 11  6 11  2 10 10 11  1 11  9  8 12  2 11 14
 10 11  0 11 14 14 14 11 13 10 11 14  1 11  6 11 11  8  3  9 15 14 11 11
 12 11 14 14  3 14  3  4  1  1 11  4 12 13 14 11 11  7  9 11 11 14 11 11
  3  3  4 11]


In [ ]:
df = [None]*branches2
dfr = [None]*branches

In [ ]:
for j in range(branches):
  print()
  dfr[j]=pd.DataFrame()
  print("Section: "+str(j+1))
  for i in range(100000):
    if reslt[i]==j:
      dfr[j]=dfr[j].append(dataset.iloc[[i]])
  print(dfr[j].head(10))


Section: 1
                                             abstract  ...                                    id
2   Recently, Bridges and Reich introduced the con...  ...  00bcf2d5-1592-46b0-81fd-933f90b5ecca
3   Most applications of the abstract interpretati...  ...  00c85316-bddf-4bcb-93f5-097adadd73c2
9                                                      ...  01546472-0ac9-47fa-acca-ffcb7fc99097
12  In system operations the term rollback is ofte...  ...  01705f09-d395-4a0f-a89f-8d18382c81cc
20                                                     ...  02d48327-6f23-4906-b2fb-c1ff66bf6b74
21  We discuss traveling wave solutions to the Yuk...  ...  034261a4-1e9b-463f-9549-c0b890af6a41
24                                                     ...  03bec38d-39fb-4bc8-be7b-9622e86ff5ff
29                                                     ...  04a259d4-1c49-4192-a1ec-c20b1fab56ec
31  We study a hypergraph transversal computation:...  ...  05440797-8533-4496-b6ea-c414a4379fea
36                

In [ ]:
for j in range(branches2):
  print()
  df[j]=pd.DataFrame()
  print("Sub-section: "+str(j+1))
  for i in range(1000):
    if fin_res[i]==j:
      df[j]=df[j].append(dataset.iloc[[i]])
  print(df[j].head(10))


Sub-section: 1
                                              abstract  ...                                    id
50                                                      ...  099d8c2e-f02a-4a67-89a3-ab5819231baf
104  The syntactic complexity of a regular language...  ...  11c8ce25-3fb2-48fa-9e18-76955fa34222
146                                                     ...  1ce4e1f6-1322-42d5-a844-108578dd599d
172  Voting is a simple mechanism to aggregate the ...  ...  21ab7cbc-12ec-4def-a407-48fbee0fcb06
175                                                     ...  22956150-5402-43a6-86d7-13c4f6123484
219  In a recent work we have proposed a comprehens...  ...  2af4e55e-8117-452a-b0b5-b274a1cc7980
252  We investigate the computational properties of...  ...  31d6d54f-48e2-4361-ab15-31c916db4cfc
253                                                     ...  31de902b-835c-4c58-9d39-c531899184e9
255                                                     ...  32267589-2b04-4aa7-b991-6a7438fc2c50
261 